In [ ]:
#NumtDB dataset
# google drive link - https://drive.google.com/file/d/1p-P5JaHM-6gaVk6soB-JV7l5l5SUoDaR/view?usp=sharing
!gdown 1p-P5JaHM-6gaVk6soB-JV7l5l5SUoDaR
!unzip /content/archive.zip

In [ ]:
# Importing necessary libraries
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import pickle
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, Activation,Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical

In [ ]:
#Declaring constants
FIG_WIDTH=20 # Width of figure
HEIGHT_PER_ROW=3 # Height of each row when showing a figure which consists of multiple rows
RESIZE_DIM=32 # The images will be resized to 28x28 pixels

In [ ]:
paths_train_a=glob.glob(os.path.join("/content/training-a",'*.png'))
paths_train_b=glob.glob(os.path.join("/content/training-b",'*.png'))
paths_train_e=glob.glob(os.path.join("/content/training-c",'*.png'))
paths_train_c=glob.glob(os.path.join("/content/training-d",'*.png'))
paths_train_d=glob.glob(os.path.join("/content/training-e",'*.png'))
paths_train_all=paths_train_a+paths_train_b+paths_train_c+paths_train_d+paths_train_e

paths_test_a=glob.glob(os.path.join("/content/testing-a",'*.png'))
paths_test_b=glob.glob(os.path.join("/content/testing-b",'*.png'))
paths_test_e=glob.glob(os.path.join("/content/testing-c",'*.png'))
paths_test_c=glob.glob(os.path.join("/content/testing-d",'*.png'))
paths_test_d=glob.glob(os.path.join("/content/testing-e",'*.png'))
paths_test_f=glob.glob(os.path.join("/content/testing-f",'*.png'))
paths_test_auga=glob.glob(os.path.join('/content/testing-auga','*.png'))
paths_test_augc=glob.glob(os.path.join('/content/testing-augc','*.png'))
paths_test_all=paths_test_a+paths_test_b+paths_test_c+paths_test_d+paths_test_e+paths_test_f+paths_test_auga+paths_test_augc

path_label_train_a='/content/training-a.csv'
path_label_train_b='/content/training-b.csv'
path_label_train_e='/content/training-c.csv'
path_label_train_c='/content/training-d.csv'
path_label_train_d='/content/training-e.csv'

In [ ]:
def get_key(path):
    # seperates the key of an image from the filepath
    key=path.split(sep=os.sep)[-1]
    return key

def get_data(paths_img,path_label=None,resize_dim=None):
    '''reads images from the filepaths, resizes them (if given), and returns them in a numpy array
    Args:
        paths_img: image filepaths
        path_label: pass image label filepaths while processing training data, defaults to None while processing testing data
        resize_dim: if given, the image is resized to resize_dim x resize_dim (optional)
    Returns:
        X: group of images
        y: categorical true labels
    '''
    X=[] # initialize empty list for resized images
    for i,path in enumerate(paths_img):
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE) # images loaded in color (BGR)
        #img = cv2.bilateralFilter(img,9,75,75)
        #img = cv2.medianBlur(img,5)
        #img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
        #img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # cnahging colorspace to GRAY
        if resize_dim is not None:
            img=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA) # resize image to 28x28
        #X.append(np.expand_dims(img,axis=2)) # expand image to 28x28x1 and append to the list.
        gaussian_3 = cv2.GaussianBlur(img, (9,9), 10.0) #unblur
        img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
        img = cv2.filter2D(img, -1, kernel)
        #thresh = 200
        #maxValue = 255
        #th, img = cv2.threshold(img, thresh, maxValue, cv2.THRESH_BINARY);
        ret,img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        X.append(img) # expand image to 28x28x1 and append to the list
        # display progress
        if i==len(paths_img)-1:
            end='\n'
        else: end='\r'
        print('processed {}/{}'.format(i+1,len(paths_img)),end=end)

    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename')
        y_label=[df.loc[get_key(path)]['digit'] for path in  paths_img] # get the labels corresponding to the images
        y=to_categorical(y_label,10) # transfrom integer value to categorical variable
        return X, y

def imshow_group(X,y,y_pred=None,n_per_row=10,phase='processed'):
    '''helper function to visualize a group of images along with their categorical true labels (y) and prediction probabilities.
    Args:
        X: images
        y: categorical true labels
        y_pred: predicted class probabilities
        n_per_row: number of images per row to be plotted
        phase: If the images are plotted after resizing, pass 'processed' to phase argument.
            It will plot the image and its true label. If the image is plotted after prediction
            phase, pass predicted class probabilities to y_pred and 'prediction' to the phase argument.
            It will plot the image, the true label, and it's top 3 predictions with highest probabilities.
    '''
    n_sample=len(X)
    img_dim=X.shape[1]
    j=int(np.ceil(n_sample/n_per_row))
    fig=plt.figure(figsize=(FIG_WIDTH,HEIGHT_PER_ROW*j))
    for i,img in enumerate(X):
        plt.subplot(j,n_per_row,i+1)
#         img_sq=np.squeeze(img,axis=2)
#         plt.imshow(img_sq,cmap='gray')
        plt.imshow(img)
        if phase=='processed':
            plt.title(np.argmax(y[i]))
        if phase=='prediction':
            top_n=3 # top 3 predictions with highest probabilities
            ind_sorted=np.argsort(y_pred[i])[::-1]
            h=img_dim+4
            for k in range(top_n):
                string='pred: {} ({:.0f}%)\n'.format(ind_sorted[k],y_pred[i,ind_sorted[k]]*100)
                plt.text(img_dim/2, h, string, horizontalalignment='center',verticalalignment='center')
                h+=4
            if y is not None:
                plt.text(img_dim/2, -4, 'true label: {}'.format(np.argmax(y[i])),
                         horizontalalignment='center',verticalalignment='center')
        plt.axis('off')
    plt.show()

def create_submission(predictions,keys,path):
    result = pd.DataFrame(
        predictions,
        columns=['label'],
        index=keys
        )
    result.index.name='key'
    result.to_csv(path, index=True)

In [ ]:
X_train_a,y_train_a=get_data(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)
X_train_b,y_train_b=get_data(paths_train_b,path_label_train_b,resize_dim=RESIZE_DIM)
X_train_c,y_train_c=get_data(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)
X_train_d,y_train_d=get_data(paths_train_d,path_label_train_d,resize_dim=RESIZE_DIM)
X_train_e,y_train_e=get_data(paths_train_e,path_label_train_e,resize_dim=RESIZE_DIM)

In [ ]:
X_train_all=np.concatenate((X_train_a,X_train_b,X_train_c,X_train_d,X_train_e),axis=0)
y_train_all=np.concatenate((y_train_a,y_train_b,y_train_c,y_train_d,y_train_e),axis=0)
X_train_all.shape, y_train_all.shape

In [ ]:
X_test_a=get_data(paths_test_a,resize_dim=RESIZE_DIM)
X_test_b=get_data(paths_test_b,resize_dim=RESIZE_DIM)
X_test_c=get_data(paths_test_c,resize_dim=RESIZE_DIM)
X_test_d=get_data(paths_test_d,resize_dim=RESIZE_DIM)
X_test_e=get_data(paths_test_e,resize_dim=RESIZE_DIM)
X_test_f=get_data(paths_test_f,resize_dim=RESIZE_DIM)
X_test_auga=get_data(paths_test_auga,resize_dim=RESIZE_DIM)
X_test_augc=get_data(paths_test_augc,resize_dim=RESIZE_DIM)

In [ ]:
X_test_all=np.concatenate((X_test_a,X_test_b,X_test_c,X_test_d,X_test_e,X_test_f,X_test_auga,X_test_augc))

In [ ]:
X_test_all.shape

In [ ]:
#train and validation split
indices=list(range(len(X_train_all)))
np.random.seed(42)
np.random.shuffle(indices)

ind=int(len(indices)*0.80)
# train data
X_train=X_train_all[indices[:ind]]
y_train=y_train_all[indices[:ind]]
# validation data
X_val=X_train_all[indices[-(len(indices)-ind):]]
y_val=y_train_all[indices[-(len(indices)-ind):]]

In [ ]:
labels = []
for i in range(len(y_train)):
  new_arrr = []
  new_arrr = (np.where(y_train[i]==1.0)[0][0])
  labels.append(new_arrr)

In [ ]:
test_labels = []
for i in range(len(y_val)):
  new_arrr = []
  new_arrr = (np.where(y_val[i]==1.0)[0][0])
  test_labels.append(new_arrr)

In [ ]:
def my_model(img_size=32,channels=1):
    model = Sequential()
    input_shape = (img_size,img_size,channels)
    model.add(Conv2D(32, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    # UNCOMMENT THIS TO VIEW THE ARCHITECTURE
    #model.summary()

    return model
model=my_model()
model.summary()

In [ ]:
path_model='model_filter.h5' # save model at this location after each epoch

model=my_model() # create the model
K.set_value(model.optimizer.lr,1e-3) # set the learning rate
# fit the model
h=model.fit(x=X_train,
            y=y_train,
            batch_size=64,
            epochs=10,
            verbose=1,
            validation_data=(X_val,y_val),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
            )

In [ ]:
predictions_prob=model.predict(X_test_all)

In [ ]:
val_pred = model.predict(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(np.argmax(y_val,axis=1), np.argmax(val_pred,axis=1))

In [ ]:
confusion

In [ ]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1', 'class 2','class 3','class 4','class 5','class 6','class 7','class 8','class 9']
print(classification_report(np.argmax(y_val,axis=1), np.argmax(val_pred,axis=1), target_names=target_names))

# NumtaDB on Ramdom forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model_random = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
X_train.shape

In [ ]:
model_random.fit(X_train.reshape(X_train.shape[0], 32*32), labels)

In [ ]:
y_predict = model_random.predict(X_val.reshape(X_val.shape[0], 32*32))

In [ ]:
y_predict

In [ ]:
y_predict

In [ ]:
# Evaluate the model's performance
accuracy = accuracy_score(test_labels, y_predict)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(test_labels, y_predict)

In [ ]:
confusion

In [ ]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1', 'class 2','class 3','class 4','class 5','class 6','class 7','class 8','class 9']
print(classification_report(test_labels, y_predict, target_names=target_names))

# NumtaDB on KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
k = 5  # Number of neighbors
model_KNN = KNeighborsClassifier(n_neighbors=k)

In [ ]:
model_KNN.fit(X_train.reshape(X_train.shape[0], 32*32), labels)

In [ ]:
y_pred_KNN = model_KNN.predict(X_val.reshape(X_val.shape[0], 32*32))

In [ ]:
accuracy = accuracy_score(test_labels, y_pred_KNN)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(test_labels, y_pred_KNN)

In [ ]:
confusion

In [ ]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1', 'class 2','class 3','class 4','class 5','class 6','class 7','class 8','class 9']
print(classification_report(test_labels, y_pred_KNN, target_names=target_names))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
confusion_matrix_data = np.array([[1301,   53,    4,   33,    1,   18,    7,   19,    5,    3],
       [  98, 1299,   26,    5,    0,    5,    7,   12,    1,   58],
       [  39,  179, 1148,    0,    2,   10,    7,   17,   14,    5],
       [ 159,   33,    8, 1003,    1,   11,  191,   17,   13,    4],
       [ 247,  132,  104,   11,  827,   44,   17,   50,   10,    4],
       [ 342,   38,   38,   97,   22,  670,  217,   16,   14,    5],
       [  75,   42,   19,  270,    1,   32,  979,    6,   22,    0],
       [  68,   36,    8,    4,    1,   16,   15, 1220,   10,    4],
       [  43,   23,   12,   11,    7,    7,   14,    6, 1297,    0],
       [ 144,  599,   52,   20,    5,   19,   23,   56,   12,  510]])

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix_data, figsize=(8, 6), cmap=plt.cm.Greens)
plt.xlabel('Predictions', fontsize=14)
plt.ylabel('Actuals', fontsize=14)
plt.title('Confusion Matrix', fontsize=14)
plt.show()